In [9]:
image_path = "ex.tif" # Path to the tif image
threshold = 1 # threshold (inclusive) for detection of a protein

In [10]:
import cv2
import numpy as np
from scipy.signal import convolve2d
from PIL import Image

In [11]:
# Test image
# image = np.array([
#     [0, 1, 1, 0, 0, 0, 0],
#     [0, 1, 0, 0, 0, 0, 1],
#     [0, 0, 0, 0, 0, 0, 0],
#     [0, 1, 0, 0, 0, 0, 0],
#     [1, 1, 0, 0, 0, 1, 0],
# ], dtype=np.uint8)

# Reading image
image = Image.open('ex.tif')
image = np.array(image)
orig_image = image.copy()
image = image >= threshold
image = image.astype(np.uint8)
image

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       ...,
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [12]:
# Defining the kernels
# The open close kernel
growing_kernel = np.array([
    [1, 1, 1],
    [1, 1, 1],
    [1, 1, 1],
], dtype=np.uint8)

# Kernel for detecting aggregates
aggregation_kernel = np.array([
    [1, 1, 1],
    [1, 0, 1],
    [1, 1, 1],
], dtype=np.uint8)

In [13]:
# Closing for filling gaps
closed = cv2.morphologyEx(image, cv2.MORPH_CLOSE, growing_kernel)

# Opening in order not to have a extended region more than the closed gaps
opened = cv2.morphologyEx(closed, cv2.MORPH_OPEN, growing_kernel)


opened

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 1, 1],
       [0, 0, 0, ..., 1, 1, 1],
       [0, 0, 0, ..., 1, 1, 1]], dtype=uint8)

In [14]:
# Detecting the aggregates
aggregates = convolve2d(image, aggregation_kernel)

aggregates = aggregates >= 1

aggregates = aggregates[1:-1, 1:-1]

aggregates.astype(np.uint8)

array([[0, 0, 1, ..., 0, 0, 0],
       [0, 0, 1, ..., 1, 1, 0],
       [0, 0, 1, ..., 0, 1, 1],
       ...,
       [0, 1, 1, ..., 1, 1, 1],
       [0, 1, 1, ..., 1, 0, 1],
       [0, 0, 1, ..., 1, 1, 1]], dtype=uint8)

In [15]:
# Removing aggregates
filtered_image = orig_image.copy()
filtered_image[aggregates] = 0

filtered_image

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [16]:
np.max(filtered_image)

4.0